# Introduction to Convolutional Operations using Numpy

## The Convolutional Operation Mathematically

Mathematically, the discrete convolution operation performed on two functions defined on finite sets of integers is formulated as follows:

$$(f * g)[n] = \sum_{m=-M}^M f[n-m]g[m]$$

where $g$ has finite support in the set $\{-M, -M+1, \ldots, M-1, M\}$.

Let's implement this (in an unoptimized form) using Python with functions $f: [0, 1, 2, 3, 4] \mapsto [1, 2, -1, 1, -3]$, and $g: [-1, 0, 1] \mapsto [-1, 0, 1]$. In order for the convolution function to work for every position of $f$, we first pad $f$ with zeroes at position -1 and 5.

In [53]:
from infix import or_infix as infix

f = {-1: 0, 0: 1, 1: 2, 2: -1, 3: 1, 4: -3, 5: 0}
g = {-1: -1, 0: 0, 1: 1}

@infix
def cnv(f, g):
    def conv_func(n):
        M = int(len(g)/2)
        res = 0
        for m in range(-M, M+1, 1):
            res += f[n - m] * g[m]
        return res
    return conv_func

for n in [0, 1, 2, 3, 4]:
    print((f |cnv| g)(n))

-2
2
1
2
1


So in words, the convolution operator first reverses the order of the elements in $g$ and then calculates the sum of the element-wise product of a $g$-sized window of $f$ and the reversed elements of $g$, at every position in $f$. Note however, that in machine learning applications, one usually doesn't bother first reversing the elements of $g$.

## The Dot Product

Summing over element-wise products is rarely being done as in the code above, but is usually implemented more efficiently with dot products. Let's illustrate this with two vectors of the same length, $\mathbf{a} = [a_1, a_2, \ldots, a_n]$, and $\mathbf{b} = [b_1, b_2, \ldots, b_n]$:

$$ \mathbf{a} \cdot \mathbf{b} = \mathbf{a}\mathbf{b}^\top = \sum_i^n a_i b_i = a_1 b_1 + a_2 b_2 + \ldots + a_n b_n$$

In [1]:
import numpy as np
a = np.array([[0, 1, 2]])
b = np.array([[1, 0, -1]])
print(np.dot(a, b.T))

[[-2]]


## Convolutions Implemented Efficiently

However with the dot product, we only got rid of one for-loop in our initial implementation of the convolutional operation, as the dot product still needs to be applied on all positions $n$ in the function $f$. Instead of applying the dot products separately for every position, the convolution operation can be vectorized in a single matrix multiplication. Let's first see the simplest situation, with convolutions in one spatial dimension, and with only 1 feature.

### One Spatial Dimension, One Feature

Consider an $n \times 1$ feature matrix $\mathbf{X}$ with elements $x_{i}$, where $i \in \{1, 2, \ldots, n\}$ is not arbitrary but represents a certain ordering or position, and a $k \times 1$ filter $\mathbf{W}$ where $k \leq n$, as in the left part of the figure below (with stride 1). Note in the figure below, that convolutions decrease the length of the feature matrix by $k - 1$, if strides of 1 are used.

![image.png](http://cs231n.github.io/assets/cnn/stride.jpeg)

In [2]:
X = np.transpose(np.array([[0, 1, 2, -1, 1, -3, 0]]))
print(X)
print(X.shape)

[[ 0]
 [ 1]
 [ 2]
 [-1]
 [ 1]
 [-3]
 [ 0]]
(7, 1)


In [3]:
W = np.transpose(np.array([[1, 0, -1]]))
print(W)
print(W.shape)

[[ 1]
 [ 0]
 [-1]]
(3, 1)


Now we will create a new feature matrix $\hat{\mathbf{X}}$ with shape $(n - k + 1) \times k$ in which the rows are the vector for which we need to perform the dot product with the filter $\mathbf{W}$. We can do this with fancy indexing like this.

In [4]:
ix = np.array([[0, 1, 2], 
               [1, 2, 3], 
               [2, 3, 4],
               [3, 4, 5],
               [4, 5, 6]])
Xhat = np.squeeze(X[ix])
print(Xhat)
print(Xhat.shape)

[[ 0  1  2]
 [ 1  2 -1]
 [ 2 -1  1]
 [-1  1 -3]
 [ 1 -3  0]]
(5, 3)


Now the matrix multiplication $\mathbf{Z} = \hat{\mathbf{X}}\mathbf{W}$ yields the convolution as a $(n - k + 1) \times 1$ matrix.

In [5]:
Z = np.dot(Xhat, W)
print(Z)
print(Z.shape)

[[-2]
 [ 2]
 [ 1]
 [ 2]
 [ 1]]
(5, 1)


### One Spatial Dimension, Multiple Features (Channels)

The above can be extended to more features, also called channels: now $\mathbf{X}$ is a $n \times d$ matrix, where $d$ is the number of features. 

When there are multiple features, the convolution operation doesn't only aggregate along the spatial dimension, but does so over the feature dimension too. So the filter also needs to have an extra dimension according to the number of features: now $\mathbf{W}$ is a $k \times d$ matrix.

In [6]:
X = np.transpose(np.array([[0, 1, 2, -1, 1, -3, 0],
                           [1, 3, -2, 2, 0, 1, -1]]))
print(X)
print(X.shape)

[[ 0  1]
 [ 1  3]
 [ 2 -2]
 [-1  2]
 [ 1  0]
 [-3  1]
 [ 0 -1]]
(7, 2)


In [7]:
W = np.transpose(np.array([[1, 0, -1],
                           [-1, 0, 1]]))
print(W)
print(W.shape)

[[ 1 -1]
 [ 0  0]
 [-1  1]]
(3, 2)


To vectorize the convolution with multiple features, we first need to flatten the filter to a $kd \times 1$ column matrix.

In [8]:
W = W.reshape((6, 1))
print(W)
print(W.shape)

[[ 1]
 [-1]
 [ 0]
 [ 0]
 [-1]
 [ 1]]
(6, 1)


With fancy indexing we create a new $(n - k + 1) \times kd$ feature matrix $\hat{\mathbf{X}}$ where the elements in every row correspond with the right weights in the flattened filter.

In [9]:
ps_ix = np.array([[0, 0, 1, 1, 2, 2], 
                  [1, 1, 2, 2, 3, 3],
                  [2, 2, 3, 3, 4, 4],
                  [3, 3, 4, 4, 5, 5],
                  [4, 4, 5, 5, 6, 6]])
ft_ix = np.array([[0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1]])
Xhat = X[ps_ix, ft_ix]
print(Xhat)
print(Xhat.shape)

                  

[[ 0  1  1  3  2 -2]
 [ 1  3  2 -2 -1  2]
 [ 2 -2 -1  2  1  0]
 [-1  2  1  0 -3  1]
 [ 1  0 -3  1  0 -1]]
(5, 6)


Now the matrix multiplication $\mathbf{Z} = \hat{\mathbf{X}}\mathbf{W}$ still yields the convolution as a $(n - k + 1) \times 1$ matrix, as the convolution has aggregated across both spatial and feature dimensions.

In [ ]:
out = np.dot(Xhat, W)
print(out)
print(out.shape)

NameError: name 'Xrow' is not defined

### Multiple Spatial Dimensions, One Feature

Let's now consider the situation where the convolution happens in two dimensions instead of one. Now we have an $n_h \times n_w$ feature matrix $\mathbf{X}$, and a $k_h \times k_w$ filter $\mathbf{W}$. Now on every convolution position, the feature values within a window the same size of the filter are aggregated to one value, and the positions can change in two directions, as illustrated in the figure below.

![image2.png](http://colah.github.io/posts/2014-07-Understanding-Convolutions/img/RiverTrain-ImageConvDiagram.png)

In [11]:
X = np.random.randint(-3, 3, size=(5, 5))
print(X)
print(X.shape)

[[ 2  1 -1 -1  2]
 [-1  1  0 -2  1]
 [-1  0 -1  0 -1]
 [-3 -2 -1  1 -1]
 [-1 -3 -2 -3 -3]]
(5, 5)


In [12]:
W = np.random.randint(-3, 3, size=(3,3))
print(W)
print(W.shape)

[[-2  2  0]
 [ 2  2  2]
 [-3 -1 -1]]
(3, 3)


To deal with the spatial dimensions, we can again flatten the filter to a $k_h k_w \times 1$ column matrix. Then later the results will be reshaped back to the original spatial shape.

In [13]:
W = W.reshape((9, 1))
print(W)
print(W.shape)

[[-2]
 [ 2]
 [ 0]
 [ 2]
 [ 2]
 [ 2]
 [-3]
 [-1]
 [-1]]
(9, 1)


Now the fancy indexing of the feature matrix is a bit more complex. The feature matrix will be transformed to a $(n_h - k_h + 1)(n_w - k_w + 1) \times k_h k_w$ feature matrix $\hat{\mathbf{X}}$. The elements in every row correspond with the elements in the flattened filter, and the rows correspond with all possible positions in $h$ and $w$ directions. The $h$ and $w$ indexes need to be constructed in a way so that the original spatial dimensions can be easily obtained by reshaping afterwards.

In [14]:
h_ix = np.array([[0, 0, 0, 1, 1, 1, 2, 2, 2],
                 [0, 0, 0, 1, 1, 1, 2, 2, 2],
                 [0, 0, 0, 1, 1, 1, 2, 2, 2],
                 [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 [1, 1, 1, 2, 2, 2, 3, 3, 3],
                 [2, 2, 2, 3, 3, 3, 4, 4, 4],
                 [2, 2, 2, 3, 3, 3, 4, 4, 4],
                 [2, 2, 2, 3, 3, 3, 4, 4, 4]])
w_ix = np.array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
                 [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 [2, 3, 4, 2, 3, 4, 2, 3, 4],
                 [0, 1, 2, 0, 1, 2, 0, 1, 2],
                 [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 [2, 3, 4, 2, 3, 4, 2, 3, 4],
                 [0, 1, 2, 0, 1, 2, 0, 1, 2],
                 [1, 2, 3, 1, 2, 3, 1, 2, 3],
                 [2, 3, 4, 2, 3, 4, 2, 3, 4]])
Xhat = X[h_ix, w_ix]
print(Xhat)
print(Xhat.shape)

[[ 2  1 -1 -1  1  0 -1  0 -1]
 [ 1 -1 -1  1  0 -2  0 -1  0]
 [-1 -1  2  0 -2  1 -1  0 -1]
 [-1  1  0 -1  0 -1 -3 -2 -1]
 [ 1  0 -2  0 -1  0 -2 -1  1]
 [ 0 -2  1 -1  0 -1 -1  1 -1]
 [-1  0 -1 -3 -2 -1 -1 -3 -2]
 [ 0 -1  0 -2 -1  1 -3 -2 -3]
 [-1  0 -1 -1  1 -1 -2 -3 -3]]
(9, 9)


Now the dot product will yield a $(n_h - k_h + 1)(n_w - k_w + 1) \times 1$ column matrix, which should be reshaped to a $(n_h - k_h + 1) \times (n_w - k_w + 1)$ matrix.

In [17]:
out = np.dot(Xhat, W)
print(out)
print(out.reshape((3, 3)))

[[ 2]
 [-5]
 [ 2]
 [12]
 [ 2]
 [-5]
 [-2]
 [ 8]
 [12]]
[[ 2 -5  2]
 [12  2 -5]
 [-2  8 12]]


### Multiple Spatial Dimensions, Multiple Features (Channels)

We can again extend this to multiple features. Now we have a $n_h \times n_w \times d$ feature matrix and a $k_h \times k_w \times d$ filter.

In [18]:
X = np.random.randint(-3, 3, (5, 5, 2))
print(X.transpose((2, 0, 1))) # transposed for display
print(X.shape)

[[[ 1 -3 -1 -2 -2]
  [-2  1  0 -1  0]
  [-2  1  0 -3 -2]
  [-2  2 -2 -3  2]
  [ 2 -2  0 -3  1]]

 [[-3  0  2 -2 -2]
  [-2  0 -3 -1  0]
  [-1 -3 -2  1 -3]
  [ 1  1 -2  2 -1]
  [ 0  2 -2  1 -2]]]
(5, 5, 2)


In [19]:
W = np.random.randint(-3, 3, (3, 3, 2))
print(W.transpose((2, 0, 1))) # transposed for display
print(W.shape)

[[[-3 -1 -1]
  [ 1  1  0]
  [-2  0  1]]

 [[ 1 -3  0]
  [ 2  1 -1]
  [ 2  0  0]]]
(3, 3, 2)


The filter needs to be flattened to a $k_h k_w d \times 1$ column matrix.

In [20]:
W = W.reshape((18, 1))
print(W)
print(W.shape)

[[-3]
 [ 1]
 [-1]
 [-3]
 [-1]
 [ 0]
 [ 1]
 [ 2]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [-2]
 [ 2]
 [ 0]
 [ 0]
 [ 1]
 [ 0]]
(18, 1)


We use fancy indexing on the feature matrix to a transformed it to a $(n_h - k_h + 1)(n_w - k_w + 1) \times k_h k_w d$ feature matrix, where elements in every row correspond with the elements in the weight column matrix.

In [21]:
h_ix = np.array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2], 
                 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2], 
                 [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2], 
                 [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3], 
                 [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3], 
                 [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3], 
                 [2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4], 
                 [2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4],
                 [2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]])
w_ix = np.array([[0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2], 
                 [1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3], 
                 [2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4],
                 [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2], 
                 [1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3], 
                 [2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4],
                 [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2], 
                 [1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3], 
                 [2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4, 2, 2, 3, 3, 4, 4]])
ft_ix = np.array([[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]])
Xhat = X[h_ix, w_ix, ft_ix]
print(Xhat)
print(Xhat.shape)

[[ 1 -3 -3  0 -1  2 -2 -2  1  0  0 -3 -2 -1  1 -3  0 -2]
 [-3  0 -1  2 -2 -2  1  0  0 -3 -1 -1  1 -3  0 -2 -3  1]
 [-1  2 -2 -2 -2 -2  0 -3 -1 -1  0  0  0 -2 -3  1 -2 -3]
 [-2 -2  1  0  0 -3 -2 -1  1 -3  0 -2 -2  1  2  1 -2 -2]
 [ 1  0  0 -3 -1 -1  1 -3  0 -2 -3  1  2  1 -2 -2 -3  2]
 [ 0 -3 -1 -1  0  0  0 -2 -3  1 -2 -3 -2 -2 -3  2  2 -1]
 [-2 -1  1 -3  0 -2 -2  1  2  1 -2 -2  2  0 -2  2  0 -2]
 [ 1 -3  0 -2 -3  1  2  1 -2 -2 -3  2 -2  2  0 -2 -3  1]
 [ 0 -2 -3  1 -2 -3 -2 -2 -3  2  2 -1  0 -2 -3  1  1 -2]]
(9, 18)


We now have a $(n_h - k_h + 1)(n_w - k_w + 1) \times k_h k_w d$ feature matrix, and a $k_h k_w d \times 1$ filter. The dot product yields a $(n_h - k_h + 1) (n_w - k_w + 1) \times 1$ column matrix, which again should be reshaped to a $(n_h - k_h + 1) \times (n_w - k_w + 1)$ matrix.

In [22]:
out = np.dot(Xhat, W)
print(out)
print(out.reshape((3, 3)))

[[-2]
 [-6]
 [ 1]
 [ 3]
 [-6]
 [ 0]
 [14]
 [ 6]
 [-9]]
[[-2 -6  1]
 [ 3 -6  0]
 [14  6 -9]]


### Multiple filters

In the context of neural networks, usually multiple filters are applied in hidden layers, resulting in similar results as above, except that there will be an extra dimension corresponding with the number of filters, which after passing through an activation function will serve as input features for the next neural network layer. In this case the filter matrix is $k_h \times k_w \times d) \times o$, where $o$ is the number of filters. This is illustrated in the figure below.

![image3.gif](https://cdn-images-1.medium.com/max/1600/1*_34EtrgYk6cQxlJ2br51HQ.gif)

In [23]:
W = np.random.randint(-3, 3, (3*3*2, 2))
print(W)
print(W.shape)

[[-1 -1]
 [-3  1]
 [ 0  2]
 [ 2  1]
 [-1  2]
 [-1 -2]
 [ 2  1]
 [-1  0]
 [-3 -1]
 [ 1  0]
 [ 2 -1]
 [-2 -2]
 [-3  2]
 [-3 -2]
 [-1 -2]
 [-2 -3]
 [-2 -2]
 [ 2 -2]]
(18, 2)


In [24]:
out = np.dot(Xhat, W)
print(out)
print(out.shape)

out = out.reshape((3, 3, 2))
np.transpose(out, (2, 0, 1)) # transposed for display

[[ 18  -4]
 [ 28  25]
 [  5  15]
 [  5   4]
 [ -3  12]
 [ 26   3]
 [-21  12]
 [ 15  -9]
 [ 29   5]]
(9, 2)


array([[[ 18,  28,   5],
        [  5,  -3,  26],
        [-21,  15,  29]],

       [[ -4,  25,  15],
        [  4,  12,   3],
        [ 12,  -9,   5]]])